# HER2 One Scanner - Aperio FDA

- 3-Fold (50/50) split, No Holdout Set
- Truth = Categorical from Mean of 7 continuous scores 
- Epoch at automatic Stop when loss<.001 change 
- LeNet model, 10 layers, Dropout (0.7)

In [60]:
import numpy as np
import pandas as pd
import random
from keras.callbacks import EarlyStopping
from PIL import Image
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, Lambda
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_curve, auc, classification_report
import csv
import cv2
import scipy
import os
%matplotlib inline
import matplotlib.pyplot as plt

In [61]:
#For single scanner
BASE_PATH = '/home/diam/Desktop/HER2_data_categorical/Aperio_FDA/'
#BASE PATH for working from home:
#BASE_PATH = '/home/OSEL/Desktop/HER2_data_categorical/'
#epochs = 10
batch_size = 32
num_classes = 3
#epochs = 35

## Get Data - Practice

In [62]:
#This is the version from Ravi's code:

#FDA
#X_FDA = []
#idx_FDA = []
#for index, image_filename in list(enumerate(BASE_PATH)):
#	img_file = cv2.imread(BASE_PATH + '/' + image_filename)
#	if img_file is not None:
		#img_file = smisc.imresize(arr = img_file, size = (600,760,3))
#		img_file = smisc.imresize(arr = img_file, size = (120,160,3))		
#		img_arr = np.asarray(img_file)
#		X_FDA.append(img_arr)
#		idx_FDA.append(index)

#X_FDA = np.asarray(X_FDA)
#idx_FDA = np.asarray(idx_FDA)
#random.seed(rs)
#random_id = random.sample(idx_FDA, len(idx_FDA)/2)
#random_FDA = []
#for i in random_id:
#	random_FDA.append(X_FDA[i])

#random_FDA = np.asarray(random_FDA)

## Get Data - Real

In [63]:
def get_data(folder):
    X = []
    y = []
    filenames = []

    for hclass in os.listdir(folder):
        if not hclass.startswith('.'):
            if hclass in ["1"]:
                label = 1
            else: #label must be 1 or 2
                if hclass in ["2"]:
                    label = 2
                else:
                    label = 3
            for image_filename in os.listdir(folder + hclass):
                filename = folder + hclass + '/' + image_filename
                img_file = cv2.imread(folder + hclass + '/' + image_filename)
                
                if img_file is not None:
                    img_file = scipy.misc.imresize(arr=img_file, size=(120, 160, 3))
                    img_arr = np.asarray(img_file)
                    X.append(img_arr)
                    y.append(label)
                    filenames.append(filename)

    X = np.asarray(X)
    y = np.asarray(y)
    z = np.asarray(filenames)
    return X,y,z

In [64]:
X, y, z = get_data(BASE_PATH)


#print(X)
#print(y)
#print(z)
print(len(X))
print(len(y))
print(len(z))

#INTEGER ENCODE
#https://machinelearningmastery.com/how-to-one-hot-encode-sequence-data-in-python/
encoder = LabelEncoder()
y_cat = np_utils.to_categorical(encoder.fit_transform(y))
#print(y_cat)





/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


241
241
241


### Old Code

In [65]:

#encoder = LabelEncoder()
#encoder.fit(y)

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

#encoded_y_train = encoder.transform(y_train)
#encoded_y_test = encoder.transform(y_test)

#y_train = np_utils.to_categorical(encoded_y_train)
#y_test = np_utils.to_categorical(encoded_y_test)

In [66]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

## Fit Model with K-Fold X-Val

In [67]:
#Need to remove random_state before running multiple thousands of times and averaging
kf = KFold(n_splits = 2, random_state=5, shuffle=True)
print(kf.get_n_splits(y))
print(kf)


for train_index_full, test_index in kf.split(y):
    val_index = np.random.choice(train_index_full, 21, replace=False)
    train_index = [x for x in train_index_full if x not in val_index]
    X_train, X_val, X_test = X[train_index], X[val_index], X[test_index]
    y_train, y_val, y_test = y[train_index], y[val_index], y[test_index]
    z_train, z_val, z_test = z[train_index], z[val_index], z[test_index]

2
KFold(n_splits=2, random_state=5, shuffle=True)


In [68]:
len(train_index)
#len(test_index)

100

In [71]:

oos_y = []
oos_pred = []
fold = 0


for train_index_full, test_index in kf.split(y_cat):
    fold+=1
    print("fold #{}".format(fold))
    
    random.shuffle(train_index_full)
    random.shuffle(test_index)
    
    val_number = np.rint(len(train_index_full)*0.1)
    print(val_number, type(val_number))
    val_index = np.random.choice(train_index_full, int(val_number), replace=False)
    train_index = [x for x in train_index_full if x not in val_index]
    X_train, X_val, X_test = X[train_index], X[val_index], X[test_index]
    y_train, y_val, y_test = y_cat[train_index], y_cat[val_index], y_cat[test_index]
    z_train, z_val, z_test = z[train_index], z[val_index], z[test_index]
    
    #X_train_full = X[train]
    #y_train_full = y_cat[train]
    #X_test = X[test]
    #y_test = y_cat[test]
    
    datagen = ImageDataGenerator(
        fill_mode='constant',
        horizontal_flip=True,
        vertical_flip=True,
        )
    
    #for _ in datagen.flow(y_test):
    #    if y_test[]
    
    model = Sequential()
    model.add(Lambda(lambda x: x * 1./255., input_shape=(120, 160, 3), output_shape=(120, 160, 3)))
    model.add(Conv2D(32, (3, 3), input_shape=(120, 160, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.7))
    model.add(Dense(3))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                optimizer='rmsprop',
                metrics=['accuracy'])
    
    monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=25, verbose=1, mode='auto')
   
    
    model.fit(
            X_train,
            y_train,
            validation_data=(X_val,y_val),
            callbacks=[monitor],
            shuffle=True,
            batch_size=batch_size,
            verbose=1,
            epochs=1000)
        
    pred = model.predict(X_test)
        
    oos_y.append(y_test)
    #pred = np.argmax(pred)
    predx = np.argmax(pred,axis=1)
    oos_pred.append(predx)
    
    #measure the fold's accuracy
    y_compare = np.argmax(y_test,axis=1) #for accuracy calculation
    score = metrics.accuracy_score(y_compare, predx)
    print("Fold Score (accuracy): {}".format(score))
    
    #Create confusion matrix
    
    print(y_test)
    #print(test)
    print(pred)
    print(predx)
    #print(z[test])
    #print(oos_y)
    
    
    #outlist = []
    
    #outline = index[0] + '\t' + str(z[test]) + '\t' + str(int(pred[0])) + '\t' + str(int(pred[1])) + '\t' + str(int(pred[2])) + '\t' + str(int(y[test])) + '\t' + str(predx) + \n'
    #outlist.append(outline)
    #with open("results.txt", "w") as f:
        #f.writelines(outlist)
        #f.writelines(z[test])

fold #1
(12.0, <type 'numpy.float64'>)
Train on 108 samples, validate on 12 samples
Epoch 1/1000
108/108 [==============================] - 1s 7ms/step - loss: 1.7085 - acc: 0.4259 - val_loss: 1.2404 - val_acc: 0.1667
Epoch 2/1000
108/108 [==============================] - 0s 2ms/step - loss: 0.9388 - acc: 0.5741 - val_loss: 1.2037 - val_acc: 0.1667
Epoch 3/1000
108/108 [==============================] - 0s 1ms/step - loss: 1.1955 - acc: 0.4630 - val_loss: 0.8708 - val_acc: 0.9167
Epoch 4/1000
108/108 [==============================] - 0s 2ms/step - loss: 0.9081 - acc: 0.6296 - val_loss: 1.0612 - val_acc: 0.1667
Epoch 5/1000
108/108 [==============================] - 0s 1ms/step - loss: 0.9375 - acc: 0.6111 - val_loss: 0.5679 - val_acc: 0.8333
Epoch 6/1000
108/108 [==============================] - 0s 2ms/step - loss: 0.9986 - acc: 0.5648 - val_loss: 0.6468 - val_acc: 0.9167
Epoch 7/1000
108/108 [==============================] - 0s 1ms/step - loss: 0.8502 - acc: 0.7037 - val_loss: 0.6

108/108 [==============================] - 0s 2ms/step - loss: 0.7635 - acc: 0.7315 - val_loss: 0.4107 - val_acc: 0.8333
Epoch 62/1000
108/108 [==============================] - 0s 1ms/step - loss: 0.3103 - acc: 0.9074 - val_loss: 0.2952 - val_acc: 0.9167
Epoch 63/1000
108/108 [==============================] - 0s 2ms/step - loss: 0.3210 - acc: 0.8981 - val_loss: 0.4354 - val_acc: 0.8333
Epoch 64/1000
108/108 [==============================] - 0s 1ms/step - loss: 0.2617 - acc: 0.9167 - val_loss: 0.3204 - val_acc: 0.8333
Epoch 65/1000
108/108 [==============================] - 0s 2ms/step - loss: 0.3105 - acc: 0.9074 - val_loss: 0.4206 - val_acc: 0.8333
Epoch 66/1000
108/108 [==============================] - 0s 1ms/step - loss: 0.3054 - acc: 0.9074 - val_loss: 0.2575 - val_acc: 0.9167
Epoch 67/1000
108/108 [==============================] - 0s 2ms/step - loss: 0.3090 - acc: 0.8889 - val_loss: 0.3495 - val_acc: 0.8333
Epoch 68/1000
108/108 [==============================] - 0s 1ms/step 

108/108 [==============================] - 0s 1ms/step - loss: 0.2780 - acc: 0.9167 - val_loss: 0.2375 - val_acc: 0.8333
Epoch 122/1000
108/108 [==============================] - 0s 1ms/step - loss: 0.1515 - acc: 0.9167 - val_loss: 0.3013 - val_acc: 0.8333
Epoch 123/1000
108/108 [==============================] - 0s 2ms/step - loss: 0.1151 - acc: 0.9352 - val_loss: 0.3687 - val_acc: 0.8333
Epoch 124/1000
108/108 [==============================] - 0s 2ms/step - loss: 0.0976 - acc: 0.9537 - val_loss: 0.6122 - val_acc: 0.8333
Epoch 125/1000
108/108 [==============================] - 0s 1ms/step - loss: 0.0892 - acc: 0.9537 - val_loss: 0.3209 - val_acc: 0.8333
Epoch 126/1000
108/108 [==============================] - 0s 2ms/step - loss: 0.0882 - acc: 0.9722 - val_loss: 0.2789 - val_acc: 0.8333
Epoch 127/1000
108/108 [==============================] - 0s 1ms/step - loss: 0.0765 - acc: 0.9722 - val_loss: 0.4980 - val_acc: 0.8333
Epoch 128/1000
108/108 [==============================] - 0s 2m

Train on 109 samples, validate on 12 samples
Epoch 1/1000
109/109 [==============================] - 1s 8ms/step - loss: 1.5897 - acc: 0.3761 - val_loss: 1.0453 - val_acc: 0.2500
Epoch 2/1000
109/109 [==============================] - 0s 2ms/step - loss: 1.0754 - acc: 0.4128 - val_loss: 0.9795 - val_acc: 0.6667
Epoch 3/1000
109/109 [==============================] - 0s 1ms/step - loss: 0.9995 - acc: 0.5138 - val_loss: 0.8803 - val_acc: 0.6667
Epoch 4/1000
109/109 [==============================] - 0s 2ms/step - loss: 0.9267 - acc: 0.5046 - val_loss: 0.8618 - val_acc: 0.6667
Epoch 5/1000
109/109 [==============================] - 0s 1ms/step - loss: 0.8161 - acc: 0.6789 - val_loss: 9.2927 - val_acc: 0.2500
Epoch 6/1000
109/109 [==============================] - 0s 2ms/step - loss: 2.7744 - acc: 0.6239 - val_loss: 0.7428 - val_acc: 0.7500
Epoch 7/1000
109/109 [==============================] - 0s 2ms/step - loss: 0.6571 - acc: 0.8073 - val_loss: 0.5544 - val_acc: 0.9167
Epoch 8/1000
109/

109/109 [==============================] - 0s 1ms/step - loss: 0.5945 - acc: 0.7706 - val_loss: 0.3747 - val_acc: 0.9167
Epoch 62/1000
109/109 [==============================] - 0s 2ms/step - loss: 0.3254 - acc: 0.8899 - val_loss: 0.5778 - val_acc: 0.9167
Epoch 63/1000
109/109 [==============================] - 0s 1ms/step - loss: 0.3298 - acc: 0.8899 - val_loss: 0.4069 - val_acc: 0.9167
Epoch 64/1000
109/109 [==============================] - 0s 2ms/step - loss: 0.2865 - acc: 0.8991 - val_loss: 0.3526 - val_acc: 0.9167
Epoch 00064: early stopping
Fold Score (accuracy): 0.825
[[ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]
 [ 0.  1.  0.]
 [ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 1.  0.  0.]
 [ 0.  0.  1.]
 [ 0.  0.  1

In [ ]:
print(y_train)
boolean = y_train[:,0] == 1
print(boolean)

In [ ]:
import cv2 as cv2

In [ ]:
f = cv2.imread('/home/diam/Desktop/HER2_data_aug_0/FDA_0_aug/aug_8_7339873.tif')



In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(f)

In [ ]:
fhsv = cv2.cvtColor(f,cv2.COLOR_BGR2HSV)

In [ ]:
fhsv > im.tif

In [ ]:
cv2.imwrite('imtest.tif', fhsv)

In [ ]:
from libtiff import TIFF